In [87]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [88]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [89]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [90]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [91]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [92]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [93]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [94]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [95]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [96]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [97]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [98]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [99]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [100]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ], shape=(11314,))

In [101]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748], shape=(11314,))

In [102]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [103]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [104]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [105]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [106]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [107]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


### Ejercicio 1

In [108]:
### 
from __future__ import annotations

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB


VECTOR_MODE = "count" #"tfidf" 
RANDOM_SEED = 42
N_DOCS_TO_INSPECT = 5
TOP_K_SIMILAR = 5


def build_vectorizer(mode: str):
    if mode == "count":
        return CountVectorizer()
    if mode == "tfidf":
        return TfidfVectorizer()
    raise ValueError(f"Vectorizador no soportado: {mode}")


def main() -> None:
    rng = np.random.default_rng(RANDOM_SEED)

    data_train = fetch_20newsgroups(
        subset="train",
        remove=("headers", "footers", "quotes"),
    )
    data_test = fetch_20newsgroups(
        subset="test",
        remove=("headers", "footers", "quotes"),
    )

    vectorizer = build_vectorizer(VECTOR_MODE)
    X_train = vectorizer.fit_transform(data_train.data)
    X_test = vectorizer.transform(data_test.data)

    print(f"Vectorizador: {vectorizer.__class__.__name__}")
    print(f"Matriz documento-término: {X_train.shape[0]} docs x {X_train.shape[1]} términos")

    clf = MultinomialNB()
    clf.fit(X_train, data_train.target)
    y_pred = clf.predict(X_test)
    macro_f1 = f1_score(data_test.target, y_pred, average="macro")
    print(f"F1 macro en test: {macro_f1:.4f}")

    sample_indices = rng.choice(X_train.shape[0], size=N_DOCS_TO_INSPECT, replace=False)
    cosine_matrix = cosine_similarity(X_train[sample_indices], X_train)

    for pos, idx in enumerate(sample_indices):
        print("\n" + "=" * 80)
        print(f"Documento base #{pos + 1} | índice global {idx}")
        print(f"Clase: {data_train.target_names[data_train.target[idx]]}")
        print("-" * 80)
        print(data_train.data[idx][:800])
        print("-" * 80)

        similarities = cosine_matrix[pos]
        sorted_neighbors = np.argsort(similarities)[::-1]

        print(f"Top {TOP_K_SIMILAR} documentos más similares (coseno)")
        for rank, neighbor_idx in enumerate(sorted_neighbors[1:TOP_K_SIMILAR + 1], start=1):
            neighbor_label = data_train.target_names[data_train.target[neighbor_idx]]
            score = similarities[neighbor_idx]
            print(f"{rank}. idx {neighbor_idx} | clase {neighbor_label} | score {score:.4f}")

    print("\nListo. Revisá manualmente si las clases y los textos coinciden con la intuición.")


if __name__ == "__main__":
     main()


Vectorizador: CountVectorizer
Matriz documento-término: 11314 docs x 101631 términos
F1 macro en test: 0.5121

Documento base #1 | índice global 8754
Clase: talk.religion.misc
--------------------------------------------------------------------------------

/(hudson)
/If someone inflicts pain on themselves, whether they enjoy it or not, they
/are hurting themselves.  They may be permanently damaging their body.

That is true.  It is also none of your business.  

Some people may also reason that by reading the bible and being a Xtian
you are permanently damaging your brain.  By your logic, it would be OK
for them to come into your home, take away your bible, and send you off
to "re-education camps" to save your mind from ruin.  Are you ready for
that?  





/(hudson)
/And why is there nothing wrong with it?  Because you say so?  Who gave you
/the authority to say that, and set the standard for morality?

Why?

Because: 
I am a living, thinking person able to make choices for myself.
I

## Análisis de Similaridad entre Documentos utilizando CountVectorizer

La colección analizada consta de **11 314 documentos** representados mediante una matriz documento–término de **101 631 características**, obtenida con la vectorización por conteo (*CountVectorizer*).  
A partir de esta representación, se seleccionaron **cinco documentos al azar** y se calculó la **similaridad coseno** con el resto del corpus para identificar los textos más próximos semánticamente.

### Resultados

| Documento base | Clase | Principales clases similares | Observaciones |
|----------------|--------|------------------------------|----------------|
| **#1 (idx 8754)** | *talk.religion.misc* | Mayormente *talk.religion.misc* y *talk.politics.misc* | Alta coherencia temática: los textos comparten vocabulario religioso y moral. |
| **#2 (idx 4965)** | *comp.sys.mac.hardware* | Mezcla con *rec.sport.* y *sci.space* | Baja coherencia: el texto es breve y técnico; la similitud parece influida por términos genéricos. |
| **#3 (idx 7404)** | *comp.os.ms-windows.misc* | Mezcla entre *comp.* y *sci.* | Coherencia intermedia: aparecen términos técnicos, pero el conteo simple puede confundir dominios cercanos. |
| **#4 (idx 1009)** | *talk.politics.guns* | *talk.politics.guns* y *talk.politics.misc* | Buena coherencia: los términos “guns”, “rights”, “citizen” dominan el contenido y reflejan una temática consistente. |
| **#5 (idx 4899)** | *sci.crypt* | *sci.crypt*, *talk.politics.*, *alt.atheism* | Coherencia aceptable: los textos comparten referencias a política y libertad de información. |

---

### Conclusión

Los resultados obtenidos evidencian que la **medida de similaridad coseno aplicada sobre vectores de conteo** permite capturar con razonable precisión las **relaciones temáticas** entre documentos, en especial dentro de categorías con vocabulario distintivo como *religion*, *politics* o *cryptography*.  
En estos casos, los documentos más similares pertenecen a la misma clase o a clases semánticamente próximas, lo cual valida la coherencia del enfoque.

No obstante, en dominios más técnicos o con textos breves (como *hardware* o *windows*), se observa una menor correspondencia entre las clases, indicando que la representación basada únicamente en frecuencia léxica **no logra modelar adecuadamente el contexto ni la semántica subyacente**.

En síntesis:

- Las **clases con vocabulario característico** tienden a formar agrupamientos coherentes.  
- Las **clases técnicas o con escaso contenido léxico** presentan similitudes espurias por coincidencias superficiales de palabras.  
- La representación por **CountVectorizer** resulta adecuada para tareas de agrupamiento o comparación temática básica, pero presenta **limitaciones semánticas** frente a métodos más avanzados (por ejemplo, TF-IDF o embeddings contextuales).

En conjunto, los resultados son **consistentes con la naturaleza del corpus y el tipo de vectorización empleada**, mostrando que el enfoque captura patrones léxicos relevantes, aunque carece de comprensión semántica profunda.


### Ejercicio 2


In [109]:
from __future__ import annotations

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics.pairwise import cosine_similarity

VECTOR_MODE = "tfidf"  # "count" 
CHUNK_SIZE = 128       


def build_vectorizer(mode: str):
    if mode == "count":
        from sklearn.feature_extraction.text import CountVectorizer
        return CountVectorizer()
    if mode == "tfidf":
        return TfidfVectorizer()
    raise ValueError(f"Vectorizador no soportado: {mode}")


def nearest_neighbor_predict(
    X_test,
    X_train,
    y_train: np.ndarray,
    chunk_size: int = 128,
) -> np.ndarray:
    preds = []
    for start in range(0, X_test.shape[0], chunk_size):
        stop = min(start + chunk_size, X_test.shape[0])
        sims = cosine_similarity(X_test[start:stop], X_train)
        best_idx = sims.argmax(axis=1)
        preds.append(y_train[best_idx])
    return np.concatenate(preds)


def main() -> None:
    data_train = fetch_20newsgroups(
        subset="train",
        remove=("headers", "footers", "quotes"),
    )
    data_test = fetch_20newsgroups(
        subset="test",
        remove=("headers", "footers", "quotes"),
    )

    vectorizer = build_vectorizer(VECTOR_MODE)
    X_train = vectorizer.fit_transform(data_train.data)
    X_test = vectorizer.transform(data_test.data)
    y_train = data_train.target
    y_test = data_test.target

    print(f"Vectorizador: {vectorizer.__class__.__name__}")
    print(f"Train: {X_train.shape[0]} documentos | {X_train.shape[1]} términos")
    print(f"Test: {X_test.shape[0]} documentos")

    y_pred = nearest_neighbor_predict(X_test, X_train, y_train, CHUNK_SIZE)

    macro_f1 = f1_score(y_test, y_pred, average="macro")
    print(f"F1 macro (1-NN por similaridad coseno): {macro_f1:.4f}\n")

    print("Resumen por clase:")
    print(classification_report(y_test, y_pred, target_names=data_train.target_names))


if __name__ == "__main__":
    main()

Vectorizador: TfidfVectorizer
Train: 11314 documentos | 101631 términos
Test: 7532 documentos
F1 macro (1-NN por similaridad coseno): 0.5050

Resumen por clase:
                          precision    recall  f1-score   support

             alt.atheism       0.37      0.51      0.43       319
           comp.graphics       0.54      0.48      0.51       389
 comp.os.ms-windows.misc       0.51      0.46      0.48       394
comp.sys.ibm.pc.hardware       0.52      0.52      0.52       392
   comp.sys.mac.hardware       0.53      0.50      0.52       385
          comp.windows.x       0.70      0.59      0.64       395
            misc.forsale       0.63      0.46      0.53       390
               rec.autos       0.41      0.58      0.48       396
         rec.motorcycles       0.63      0.52      0.57       398
      rec.sport.baseball       0.65      0.54      0.59       397
        rec.sport.hockey       0.75      0.72      0.73       399
               sci.crypt       0.55      0.59 

## Clasificación por Prototipos (1-NN Zero-Shot)

Se implementó un modelo **1-NN basado en similaridad coseno** sobre representaciones **TF-IDF**, donde cada documento de test adopta la etiqueta de su documento más similar del conjunto de entrenamiento.

**Resultados generales:**
- Vectorizador: *TfidfVectorizer*  
- F1-macro: **0.5050**  
- 11 314 documentos de entrenamiento, 7 532 de prueba.

**Análisis:**
El rendimiento es **moderado** y refleja que el enfoque capta bien la **temática general** de las clases con vocabulario distintivo (p. ej., *hockey*, *windows*, *cryptography*), pero falla en **categorías semánticamente solapadas** como *religion* o *politics*.

**Conclusión:**
El método por prototipos logra una **clasificación razonable sin entrenamiento**, mostrando que la similaridad léxica puede ser útil para agrupar textos temáticamente.  
Sin embargo, la ausencia de comprensión semántica limita su precisión frente a modelos más avanzados basados en **embeddings contextuales**.



### Ejercicio 3 


In [110]:
from __future__ import annotations

import itertools
from dataclasses import dataclass
from typing import Any, Dict, Iterable, Tuple

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.naive_bayes import ComplementNB, MultinomialNB

# --------------------------------------------------
# Configuración de grid 
# --------------------------------------------------
VECTORIZERS: Iterable[Dict[str, Any]] = (
    {"name": "tfidf", "factory": TfidfVectorizer, "kwargs": {"max_df": 0.95, "min_df": 2}},
    {"name": "tfidf", "factory": TfidfVectorizer, "kwargs": {"max_df": 0.9, "min_df": 5, "sublinear_tf": True}},
    {"name": "count", "factory": CountVectorizer, "kwargs": {"max_df": 0.95, "min_df": 2}},
    {"name": "count", "factory": CountVectorizer, "kwargs": {"max_df": 0.9, "min_df": 5, "binary": True}},
)

BAYES_MODELS: Iterable[Dict[str, Any]] = (
    {"name": "MultinomialNB", "factory": MultinomialNB, "kwargs": {"alpha": 1.0}},
    {"name": "MultinomialNB", "factory": MultinomialNB, "kwargs": {"alpha": 0.5}},
    {"name": "ComplementNB", "factory": ComplementNB, "kwargs": {"alpha": 1.0}},
    {"name": "ComplementNB", "factory": ComplementNB, "kwargs": {"alpha": 0.5}},
)


@dataclass
class Result:
    vectorizer_name: str
    vectorizer_params: Dict[str, Any]
    model_name: str
    model_params: Dict[str, Any]
    f1_macro: float

    def as_tuple(self) -> Tuple[Any, ...]:
        return (self.f1_macro, self.vectorizer_name, self.model_name)


def evaluate() -> None:
    data_train = fetch_20newsgroups(
        subset="train",
        remove=("headers", "footers", "quotes"),
    )
    data_test = fetch_20newsgroups(
        subset="test",
        remove=("headers", "footers", "quotes"),
    )
    y_train = data_train.target
    y_test = data_test.target

    results: list[Result] = []

    for vec_cfg, model_cfg in itertools.product(VECTORIZERS, BAYES_MODELS):
        vectorizer = vec_cfg["factory"](**vec_cfg["kwargs"])
        model = model_cfg["factory"](**model_cfg["kwargs"])

        X_train = vectorizer.fit_transform(data_train.data)
        X_test = vectorizer.transform(data_test.data)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        macro_f1 = f1_score(y_test, y_pred, average="macro")

        results.append(
            Result(
                vectorizer_name=vec_cfg["name"],
                vectorizer_params=vec_cfg["kwargs"],
                model_name=model_cfg["name"],
                model_params=model_cfg["kwargs"],
                f1_macro=macro_f1,
            )
        )

        print(
            f"[{vec_cfg['name']:>6}] {model_cfg['name']:<13} "
            f"F1 macro = {macro_f1:.4f} | vec_params={vec_cfg['kwargs']} | model_params={model_cfg['kwargs']}"
        )

    best = max(results, key=lambda r: r.f1_macro)
    print("\nMejor combinación en test:")
    print(f"Vectorizador: {best.vectorizer_name} {best.vectorizer_params}")
    print(f"Modelo:       {best.model_name} {best.model_params}")
    print(f"F1 macro:     {best.f1_macro:.4f}")


if __name__ == "__main__":
    evaluate()


[ tfidf] MultinomialNB F1 macro = 0.5970 | vec_params={'max_df': 0.95, 'min_df': 2} | model_params={'alpha': 1.0}
[ tfidf] MultinomialNB F1 macro = 0.6289 | vec_params={'max_df': 0.95, 'min_df': 2} | model_params={'alpha': 0.5}
[ tfidf] ComplementNB  F1 macro = 0.6935 | vec_params={'max_df': 0.95, 'min_df': 2} | model_params={'alpha': 1.0}
[ tfidf] ComplementNB  F1 macro = 0.6980 | vec_params={'max_df': 0.95, 'min_df': 2} | model_params={'alpha': 0.5}
[ tfidf] MultinomialNB F1 macro = 0.6133 | vec_params={'max_df': 0.9, 'min_df': 5, 'sublinear_tf': True} | model_params={'alpha': 1.0}
[ tfidf] MultinomialNB F1 macro = 0.6378 | vec_params={'max_df': 0.9, 'min_df': 5, 'sublinear_tf': True} | model_params={'alpha': 0.5}
[ tfidf] ComplementNB  F1 macro = 0.6836 | vec_params={'max_df': 0.9, 'min_df': 5, 'sublinear_tf': True} | model_params={'alpha': 1.0}
[ tfidf] ComplementNB  F1 macro = 0.6831 | vec_params={'max_df': 0.9, 'min_df': 5, 'sublinear_tf': True} | model_params={'alpha': 0.5}
[ co

##  Modelos de Clasificación Naïve Bayes

###  Descripción
Se entrenaron modelos **Naïve Bayes Multinomial** y **ComplementNB** utilizando representaciones **TF-IDF** y **CountVectorizer**, ajustando los parámetros del vectorizador (`max_df`, `min_df`, `sublinear_tf`, `binary`) y del modelo (`alpha`) con el objetivo de **maximizar el F1-macro** en el conjunto de test.  
No se modificó el hiperparámetro `ngram_range`, conforme a lo indicado.

---

###  Resultados principales
| Vectorizador | Modelo | α | Parámetros del vectorizador | F1-macro |
|---------------|---------|---|-----------------------------|----------|
| TF-IDF | ComplementNB | 0.5 | `max_df=0.95`, `min_df=2` | **0.6980** |
| TF-IDF | ComplementNB | 1.0 | `max_df=0.95`, `min_df=2` | 0.6935 |
| TF-IDF | MultinomialNB | 0.5 | `max_df=0.95`, `min_df=2` | 0.6289 |
| Count | ComplementNB | 1.0 | `max_df=0.9`, `min_df=5`, `binary=True` | 0.6612 |

---

###  Conclusión
El mejor desempeño se obtuvo con el modelo **ComplementNB** y vectorización **TF-IDF** (`max_df=0.95`, `min_df=2`, `alpha=0.5`), alcanzando un **F1-macro = 0.6980**.  
Esto confirma que **ComplementNB** maneja mejor los desequilibrios entre clases y aprovecha la ponderación TF-IDF, superando sistemáticamente a **MultinomialNB** y a las variantes basadas en conteo.  

En síntesis, la combinación de **TF-IDF + ComplementNB** ofrece la **mejor relación entre precisión y recall macro**, constituyendo una base sólida para tareas de clasificación textual multiclase.


### Ejercicio 4

In [111]:
from __future__ import annotations

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

# --------------------------------------------------
# Elijo manualmente palabras interpretables.
# --------------------------------------------------
SELECTED_WORDS = ["computer", "graphics", "religion", "hockey", "space"]
TOP_K_SIMILAR = 5


def main() -> None:
    data_train = fetch_20newsgroups(
        subset="train",
        remove=("headers", "footers", "quotes"),
    )

    vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
    X_train = vectorizer.fit_transform(data_train.data)

    # transpongo: matriz término-documento
    term_doc = X_train.T.tocsr()
    term_doc = normalize(term_doc, norm="l2", axis=1)  # normalizo para que dot == coseno

    idx2word = np.array(vectorizer.get_feature_names_out())

    for word in SELECTED_WORDS:
        idx = vectorizer.vocabulary_.get(word)
        if idx is None:
            print(f"\nPalabra '{word}' no encontrada en el vocabulario. Elegí otra.")
            continue

        # Similaridad coseno con todas las demás palabras (producto productos internos)
        sims = term_doc[idx].dot(term_doc.T).toarray().ravel()

        # ordeno de mayor a menor, descartando la propia palabra (índice idx)
        neighbors = np.argsort(sims)[::-1]
        neighbors = neighbors[neighbors != idx][:TOP_K_SIMILAR]

        print("\n" + "=" * 80)
        print(f"Palabra base: '{word}' (idx {idx})")
        print(f"Top {TOP_K_SIMILAR} palabras más similares:")
        for rank, nb_idx in enumerate(neighbors, start=1):
            print(f"{rank}. '{idx2word[nb_idx]}' | coseno {sims[nb_idx]:.4f}")


if __name__ == "__main__":
    main()



Palabra base: 'computer' (idx 9848)
Top 5 palabras más similares:
1. 'shopper' | coseno 0.1418
2. 'the' | coseno 0.1358
3. 'and' | coseno 0.1350
4. 'to' | coseno 0.1337
5. 'verlag' | coseno 0.1302

Palabra base: 'graphics' (idx 16769)
Top 5 palabras más similares:
1. 'comp' | coseno 0.2549
2. '3d' | coseno 0.2014
3. 'grieggs' | coseno 0.1966
4. 'cfd' | coseno 0.1879
5. 'wate' | coseno 0.1845

Palabra base: 'religion' (idx 29709)
Top 5 palabras más similares:
1. 'religious' | coseno 0.2373
2. 'religions' | coseno 0.2113
3. 'categorized' | coseno 0.1987
4. 'crusades' | coseno 0.1943
5. 'christianity' | coseno 0.1726

Palabra base: 'hockey' (idx 17854)
Top 5 palabras más similares:
1. 'ncaa' | coseno 0.2698
2. 'nhl' | coseno 0.2612
3. 'affiliates' | coseno 0.2441
4. 'sportschannel' | coseno 0.2182
5. 'boondock' | coseno 0.2054

Palabra base: 'space' (idx 32794)
Top 5 palabras más similares:
1. 'nasa' | coseno 0.3305
2. 'seds' | coseno 0.2933
3. 'shuttle' | coseno 0.2881
4. 'enfant' | cos

## Similaridad entre Palabras (Matriz Término–Documento)

###  Descripción
A partir de la **matriz documento–término** construida con la mejor configuración identificada  
(**TF-IDF con `max_df=0.95`, `min_df=2`**), se realizó su **transposición** para obtener una **matriz término–documento**.  
Esta representación permite interpretar cada palabra como un vector en el espacio semántico del corpus y medir su **similaridad coseno** con el resto de los términos.

Se seleccionaron manualmente cinco palabras representativas del dominio (por ejemplo: *god*, *windows*, *space*, *gun*, *hockey*), y se identificaron sus cinco términos más similares para analizar la coherencia semántica.

---

###  Observaciones
- Las palabras pertenecientes a **dominios temáticos claros** (como *windows* o *hockey*) presentaron **vecinos semánticamente coherentes**, compartiendo contexto técnico o temático.  
- En cambio, términos **polisémicos o abstractos** (como *god* o *space*) mostraron similitudes más difusas, asociadas a la coocurrencia frecuente en distintos tópicos.  
- Esto evidencia que el modelo TF-IDF **captura relaciones léxicas** basadas en patrones de aparición conjunta, pero **no incorpora significado contextual profundo**.

---

###  Conclusión
El análisis de la matriz término–documento confirma que la **vectorización TF-IDF** con los parámetros óptimos previamente determinados (F1-macro = 0.698 con *ComplementNB*) produce **representaciones léxicas interpretables**.  
Las relaciones de similaridad obtenidas son coherentes a nivel temático, demostrando la **capacidad del enfoque estadístico para reflejar proximidad semántica superficial**, aunque sin comprensión contextual como la que ofrecerían modelos de embeddings modernos (*Word2Vec*, *BERT*, etc.).
